<a href="https://colab.research.google.com/github/SolKidonakis/TP1NLP/blob/main/TP1NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**TRABAJO PRACTICO N°1: PROCESAMIENTO DEL LENGUAJE NATURAL**

**LIBRERIAS**

In [ ]:
!pip install sentence-transformers
!python -m spacy download es_core_news_md
!pip install nltk
!pip install ipywidgets
!pip install --upgrade tensorflow tensorflow-hub

In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import ipywidgets as widgets
from nltk.corpus import stopwords
import nltk
import spacy
from concurrent.futures import ThreadPoolExecutor, as_completed
import time
import numpy as np


/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


En primer lugar,haremos web scraping para extraer información sobre los libros de Project Gutenberg, incluyendo su título, autor, descripción,categoria y enlace, y guardamos todos esos datos en un archivo CSV para su posterior uso.

In [5]:
# Función para obtener el objeto Soup de una URL
def get_soup(url):
    response = requests.get(url)
    return BeautifulSoup(response.content, 'html.parser')

def extraer_info(book_soup, url):
    table = book_soup.find('table', class_='bibrec')
    rows = table.find_all('tr') if table else []


    title = author  = summary = None
    subjects = []

    for row in rows:
        th = row.find('th')
        td = row.find('td')

        if th and td:
            if 'Author' in th.text:
                author = td.text.strip()
            elif 'Title' in th.text:
                title = td.text.strip()
            elif 'Summary' in th.text:
                summary = td.text.strip().replace("(This is an automatically generated summary.)", "").strip('"')
            elif 'Subject' in th.text:
                subject_links = td.find_all('a')
                for link in subject_links:
                    subjects.append(link.text.strip())

    subjects_string = ", ".join(subjects)

    return {
        'Title': title,
        'Author': author,
        'Summary': summary,
        'Subjects': subjects_string,
        'URL': url
    }

# Obtener la lista de URLs de libros desde la página principal de Project Gutenberg
def libros_urls(base_url="https://www.gutenberg.org"):
    url = f"{base_url}/browse/scores/top1000.php#books-last1"
    soup = get_soup(url)
    book_elements = soup.select('ol li')
    urls = []

    for element in book_elements:
        a_tag = element.find('a')
        if a_tag and 'href' in a_tag.attrs:
            full_url = f"{base_url}{a_tag['href']}"
            urls.append(full_url)

    return urls

# Procesar cada libro con ThreadPoolExecutor
def procesar_libros(urls):
    books = []
    with ThreadPoolExecutor(max_workers=5) as executor:
        futures = [executor.submit(procesar_libro_individual, url) for url in urls]
        for future in as_completed(futures):
            result = future.result()
            if result:
                books.append(result)
            time.sleep(0.1)  # Mantener un delay entre requests
    return books


def procesar_libro_individual(url):
    try:
        book_soup = get_soup(url)
        return extraer_info(book_soup, url)
    except Exception as e:
        print(f"Error al procesar {url}: {e}")
        return None

# Ejecutar el proceso completo y guardar en un DataFrame
def main():
    urls = libros_urls()
    books = procesar_libros(urls)
    books_df = pd.DataFrame(books)
    books_df.to_csv('gutenberg_books.csv', index=False, encoding='utf-8')
    print(books_df.head())


main()


                                     Title  \
0                              Shen Yin Yu   
1                      Pride and Prejudice   
2                 Moby Dick; Or, The Whale   
3  Frankenstein; Or, The Modern Prometheus   
4                         Romeo and Juliet   

                                    Author  \
0                       Lü, Kun, 1536-1618   
1                  Austen, Jane, 1775-1817   
2              Melville, Herman, 1819-1891   
3  Shelley, Mary Wollstonecraft, 1797-1851   
4          Shakespeare, William, 1564-1616   

                                             Summary  \
0  呻吟語" by Kun Lü is a philosophical treatise wri...   
1  Pride and Prejudice" by Jane Austen is a class...   
2  Moby Dick; Or, The Whale" by Herman Melville i...   
3  Frankenstein; Or, The Modern Prometheus" by Ma...   
4  Romeo and Juliet" by William Shakespeare is a ...   

                                            Subjects  \
0                                    Conduct of life 

Abrimos tres archivos CSV que tienen información sobre juegos de mesa, películas y libros.

In [6]:
board_games_df = pd.read_csv('bgg_database.csv')

movies_df = pd.read_csv('IMDB-Movie-Data.csv')

books_df = pd.read_csv('gutenberg_books.csv')

#Imprimimos las primeras filas de cada CSV
print(board_games_df.head())
print(movies_df.head())
print(books_df.head())


   rank                          game_name  \
0     1                  Brass: Birmingham   
1     2          Pandemic Legacy: Season 1   
2     3                         Gloomhaven   
3     4                           Ark Nova   
4     5  Twilight Imperium: Fourth Edition   

                                           game_href  geek_rating  avg_rating  \
0  https://boardgamegeek.com/boardgame/224517/bra...        8.415        8.60   
1  https://boardgamegeek.com/boardgame/161936/pan...        8.377        8.53   
2  https://boardgamegeek.com/boardgame/174430/glo...        8.349        8.59   
3  https://boardgamegeek.com/boardgame/342942/ark...        8.335        8.54   
4  https://boardgamegeek.com/boardgame/233078/twi...        8.240        8.60   

   num_voters                                        description  \
0     46836.0  Brass: Birmingham is an economic strategy game...   
1     53807.0  Pandemic Legacy is a co-operative campaign gam...   
2     62592.0  Gloomhaven  is a 

Notamos que habia titulos duplicados en el dataset de Libros,por lo que decidimos quedaron con los valores unicos

In [52]:
# Eliminar duplicados en base a las columnas Title y Author
books_df = books_df.drop_duplicates(subset=['Title', 'Author'])

# Verificar la cantidad de registros únicos después de eliminar duplicados
print("Cantidad de títulos únicos después de eliminar duplicados:", books_df.shape[0])


Cantidad de títulos únicos después de eliminar duplicados: 1176


In [53]:
def limpiar_descripcion(descripcion):
    """Función para limpiar y mejorar la descripción del juego."""
    # Reemplazar caracteres HTML y eliminar etiquetas o frases irrelevantes
    descripcion = descripcion.replace('&quot;', '"').replace('&rsquo;', "'").replace('&ldquo;', '"').replace('&rdquo;', '"')
    descripcion = descripcion.replace('&mdash;', '-').replace('&ndash;', '-').replace('&lsquo;', "'").replace('&rsquo;', "'")
    descripcion = descripcion.replace('description from the publisher', '')

    # Si hay etiquetas de autor o editor en la descripción, eliminarlas
    frases_irrelevantes = ["description from the publisher", "description from", "published by", "published as"]
    for frase in frases_irrelevantes:
        descripcion = descripcion.replace(frase, '')

    # Eliminar exceso de espacios y símbolos
    descripcion = ' '.join(descripcion.split())  # Quitar espacios extra

    return descripcion


**CLASIFICADOR CON REGRESION LOGISTICA**

En este caso, utilizamos TF-IDF Vectorizer.
Usando dicho modelo de regresión logística para clasificar frases en Alegre, Melancólico o ni fu ni fa.



In [7]:
nltk.download('stopwords')

spanish_stop_words = stopwords.words('spanish')

# Etiquetas de estados de ánimo
labels = [(0, "Alegre"), (1, "Melancólico"), (2, "Ni fu ni fa")]

dataset = [
    # Frases de estado de ánimo "Alegre"
    (0, "Estoy muy feliz"),
    (0, "Hoy es un gran día"),
    (0, "Me siento alegre"),
    (0, "Es un día increíble"),
    (0, "Estoy lleno de energía"),
    (0, "Estoy muy contento hoy"),
    (0, "Este es un día perfecto"),
    (0, "Me siento lleno de vida"),
    (0, "Estoy eufórico"),
    (0, "Hoy es un día para celebrar"),
    (0, "Siento mucha alegría en este día"),
    (0, "Todo es perfecto hoy"),
    (0, "Me encanta la vida"),
    (0, "Es un día espectacular para disfrutar"),
    (0, "Hoy estoy muy entusiasmado"),
    (0, "Me siento agradecido y optimista"),
    (0, "Qué día tan brillante y hermoso"),
    (0, "Estoy exultante de felicidad"),
    (0, "Todo está saliendo a la perfección"),
    (0, "Estoy lleno de emociones positivas"),
    (0, "Siento una gran paz y alegría"),
    (0, "Hoy es un día lleno de posibilidades"),
    (0, "Nada puede arruinar mi buen humor"),
    (0, "Todo se siente más fácil y agradable"),
    (0, "Es un día maravilloso para vivir"),
    (0, "Estoy encantado con este día"),
    (0, "Me siento radiante"),
    (0, "Este día está lleno de sorpresas agradables"),
    (0, "Siento que puedo lograr cualquier cosa"),
    (0, "Hoy es un día para sonreír"),
    (0, "Estoy disfrutando cada momento"),
    (0, "Tengo una gran paz interior hoy"),
    (0, "Este día está lleno de luz"),
    (0, "Hoy estoy lleno de esperanza"),
    (0, "Todo parece más hermoso hoy"),
    (0, "Este es un día para recordar"),
    (0, "Siento que la vida es un regalo"),
    (0, "Estoy lleno de motivación y alegría"),
    (0, "Hoy es un día especial"),
    (0, "Estoy rodeado de amor y felicidad"),

    # Frases de estado de ánimo "Melancólico"
    (1, "Me siento triste"),
    (1, "Estoy melancólico"),
    (1, "Hoy estoy nostálgico"),
    (1, "Me siento deprimido"),
    (1, "No puedo dejar de pensar en cosas tristes"),
    (1, "Es un día gris"),
    (1, "Me siento vacío"),
    (1, "No tengo ánimos para nada"),
    (1, "Estoy abatido"),
    (1, "Todo me parece sombrío"),
    (1, "Siento una gran tristeza"),
    (1, "No me gusta este día"),
    (1, "Es un día muy malo"),
    (1, "Todo parece estar mal hoy"),
    (1, "Nada me da consuelo hoy"),
    (1, "Siento que todo es una carga"),
    (1, "Estoy atrapado en mis pensamientos"),
    (1, "No tengo energías para nada"),
    (1, "Es un día sin color ni esperanza"),
    (1, "Nada me hace sonreír hoy"),
    (1, "No encuentro alegría en nada"),
    (1, "Es un día de pura tristeza y soledad"),
    (1, "Siento un vacío muy profundo"),
    (1, "Nada tiene sentido hoy"),
    (1, "Me siento como una sombra"),
    (1, "Todo está en mi contra hoy"),
    (1, "Siento que estoy solo"),
    (1, "Es un día sin propósito"),
    (1, "Estoy agobiado por la tristeza"),
    (1, "Siento que me falta algo"),
    (1, "Hoy no encuentro paz"),
    (1, "Es un día para olvidar"),
    (1, "Me siento desconectado del mundo"),
    (1, "Es como si todo pesara más hoy"),
    (1, "No puedo escapar de esta sensación"),
    (1, "Este día solo trae recuerdos amargos"),
    (1, "Me siento incompleto y perdido"),
    (1, "No tengo fuerzas para continuar"),
    (1, "Es un día lleno de desesperanza"),
    (1, "Mi corazón está apesadumbrado"),
    (1, "Este día solo trae desilusión"),

    # Frases de estado de ánimo "Ni fu ni fa"
    (2, "No tengo ganas de hacer nada"),
    (2, "Me siento neutral"),
    (2, "Estoy cansado"),
    (2, "Me da igual lo que pase"),
    (2, "No me importa nada hoy"),
    (2, "Estoy indiferente"),
    (2, "No siento ni alegría ni tristeza"),
    (2, "Hoy es un día normal"),
    (2, "Estoy aburrido"),
    (2, "Nada me parece interesante"),
    (2, "No sé cómo me siento"),
    (2, "Todo está igual que siempre"),
    (2, "Nada especial sucede hoy"),
    (2, "Es un día cualquiera"),
    (2, "Hoy me siento indiferente a todo"),
    (2, "Simplemente no siento nada"),
    (2, "Es uno de esos días sin nada nuevo"),
    (2, "Ni me siento bien ni mal"),
    (2, "Nada me motiva particularmente"),
    (2, "Es un día más, sin nada que contar"),
    (2, "No tengo emociones intensas hoy"),
    (2, "Hoy no pasa nada fuera de lo común"),
    (2, "Es un día bastante monótono"),
    (2, "Todo está en el mismo estado de siempre"),
    (2, "Ni estoy triste ni contento, simplemente estoy"),
    (2, "Hoy no tengo ni energía ni ánimos"),
    (2, "Nada sobresaliente en este día"),
    (2, "Estoy en piloto automático hoy"),
    (2, "Hoy no hay nada que me anime"),
    (2, "Es un día ordinario y sin sorpresas"),
    (2, "Estoy en mi estado habitual"),
    (2, "Hoy no tengo ninguna expectativa"),
    (2, "No espero nada especial hoy"),
    (2, "Es un día como cualquier otro"),
    (2, "Nada parece fuera de lo común"),
    (2, "Hoy solo estoy existiendo"),
    (2, "No hay nada excitante en este día"),
    (2, "Me siento completamente neutral"),
    (2, "Hoy es simplemente un día más"),
    (2, "Nada parece cambiar hoy"),
]

# Preparar X (frases) e y (etiquetas de estados de ánimo)
X = [text.lower() for label, text in dataset]
y = [label for label, text in dataset]

# Dividir el dataset en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Vectorización del texto con TF-IDF, eliminando palabras vacías y usando bigramas
vectorizer = TfidfVectorizer(stop_words=spanish_stop_words, ngram_range=(1, 2))
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

# Crear y entrenar el modelo de Regresión Logística con balance de clases
modelo_LR = LogisticRegression(max_iter=1000, multi_class='multinomial', solver='lbfgs', class_weight='balanced')
modelo_LR.fit(X_train_vectorized, y_train)

# Evaluar el modelo con el conjunto de prueba
y_pred_LR = modelo_LR.predict(X_test_vectorized)
acc_LR = accuracy_score(y_test, y_pred_LR)
report_LR = classification_report(y_test, y_pred_LR, zero_division=1)

print("Precisión Regresión Logística:", acc_LR)
print("Reporte de clasificación Regresión Logística:\n", report_LR)


Precisión Regresión Logística: 0.56
Reporte de clasificación Regresión Logística:
               precision    recall  f1-score   support

           0       0.71      0.62      0.67         8
           1       0.71      0.45      0.56        11
           2       0.36      0.67      0.47         6

    accuracy                           0.56        25
   macro avg       0.60      0.58      0.56        25
weighted avg       0.63      0.56      0.57        25



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


Contamos con un accuracy del 56% ,como podemos ver el modelo no tiene muy buenas metricas.

Probamos el modelo de clasificación:

In [9]:
def predict_mood(user_input):
    user_input_vectorized = vectorizer.transform([user_input.lower()])
    mood_label = modelo_LR.predict(user_input_vectorized)[0]
    return labels[mood_label][1]


# Nuevas frases más distintas para probar el modelo
nuevas_frases = [
    "No puedo evitar sentir una gran tristeza hoy",  # Debería ser "Melancólico"
    "Siento que este es uno de los mejores días de mi vida",  # Debería ser "Alegre"
    "Hoy todo me da lo mismo, nada me interesa realmente",  # Debería ser "Ni fu ni fa"
    "No sé qué hacer, pero no estoy ni triste ni feliz",  # Debería ser "Ni fu ni fa"
    "Me siento increíblemente emocionado y lleno de energía",  # Debería ser "Alegre"
    "Todo parece gris y no veo nada que me anime",  # Debería ser "Melancólico"
    "Este día está siendo tan normal como cualquier otro",  # Debería ser "Ni fu ni fa"
    "No hay nada que me entusiasme, es como si estuviera vacío",  # Debería ser "Melancólico"
    "Estoy muy agradecido por todo, este día es fantástico",  # Debería ser "Alegre"
    "Hoy no me pasa nada en particular, no siento ni tristeza ni alegría"  # Debería ser "Ni fu ni fa"
]

# Predecir los estados de ánimo para las nuevas frases
for frase in nuevas_frases:
    print(f"La frase '{frase}' tiene el estado de ánimo: {predict_mood(frase)}")


La frase 'No puedo evitar sentir una gran tristeza hoy' tiene el estado de ánimo: Melancólico
La frase 'Siento que este es uno de los mejores días de mi vida' tiene el estado de ánimo: Alegre
La frase 'Hoy todo me da lo mismo, nada me interesa realmente' tiene el estado de ánimo: Ni fu ni fa
La frase 'No sé qué hacer, pero no estoy ni triste ni feliz' tiene el estado de ánimo: Ni fu ni fa
La frase 'Me siento increíblemente emocionado y lleno de energía' tiene el estado de ánimo: Alegre
La frase 'Todo parece gris y no veo nada que me anime' tiene el estado de ánimo: Ni fu ni fa
La frase 'Este día está siendo tan normal como cualquier otro' tiene el estado de ánimo: Ni fu ni fa
La frase 'No hay nada que me entusiasme, es como si estuviera vacío' tiene el estado de ánimo: Melancólico
La frase 'Estoy muy agradecido por todo, este día es fantástico' tiene el estado de ánimo: Alegre
La frase 'Hoy no me pasa nada en particular, no siento ni tristeza ni alegría' tiene el estado de ánimo: Ni fu

**EN ESTE CASO, UTILIZAMOS SENTENCE TRANSFORMER**

Entrenamos un modelo de clasificación (de estado de ánimo) pero en este caso, utilizando embeddings generados por el modelo de SentenceTransformer.

In [10]:

model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

labels = [(0, "Alegre"), (1, "Melancólico"), (2, "Ni fu ni fa")]


dataset = [
    # Frases de estado de ánimo "Alegre"
    (0, "Estoy muy feliz"),
    (0, "Hoy es un gran día"),
    (0, "Me siento alegre"),
    (0, "Es un día increíble"),
    (0, "Estoy lleno de energía"),
    (0, "Estoy muy contento hoy"),
    (0, "Este es un día perfecto"),
    (0, "Me siento lleno de vida"),
    (0, "Estoy eufórico"),
    (0, "Hoy es un día para celebrar"),
    (0, "Siento mucha alegría en este día"),
    (0, "Todo es perfecto hoy"),
    (0, "Me encanta la vida"),
    (0, "Es un día espectacular para disfrutar"),
    (0, "Hoy estoy muy entusiasmado"),
    (0, "Me siento agradecido y optimista"),
    (0, "Qué día tan brillante y hermoso"),
    (0, "Estoy exultante de felicidad"),
    (0, "Todo está saliendo a la perfección"),
    (0, "Estoy lleno de emociones positivas"),
    (0, "Siento una gran paz y alegría"),
    (0, "Hoy es un día lleno de posibilidades"),
    (0, "Nada puede arruinar mi buen humor"),
    (0, "Todo se siente más fácil y agradable"),
    (0, "Es un día maravilloso para vivir"),
    (0, "Estoy encantado con este día"),
    (0, "Me siento radiante"),
    (0, "Este día está lleno de sorpresas agradables"),
    (0, "Siento que puedo lograr cualquier cosa"),
    (0, "Hoy es un día para sonreír"),
    (0, "Estoy disfrutando cada momento"),
    (0, "Tengo una gran paz interior hoy"),
    (0, "Este día está lleno de luz"),
    (0, "Hoy estoy lleno de esperanza"),
    (0, "Todo parece más hermoso hoy"),
    (0, "Este es un día para recordar"),
    (0, "Siento que la vida es un regalo"),
    (0, "Estoy lleno de motivación y alegría"),
    (0, "Hoy es un día especial"),
    (0, "Estoy rodeado de amor y felicidad"),

    # Frases de estado de ánimo "Melancólico"
    (1, "Me siento triste"),
    (1, "Estoy melancólico"),
    (1, "Hoy estoy nostálgico"),
    (1, "Me siento deprimido"),
    (1, "No puedo dejar de pensar en cosas tristes"),
    (1, "Es un día gris"),
    (1, "Me siento vacío"),
    (1, "No tengo ánimos para nada"),
    (1, "Estoy abatido"),
    (1, "Todo me parece sombrío"),
    (1, "Siento una gran tristeza"),
    (1, "No me gusta este día"),
    (1, "Es un día muy malo"),
    (1, "Todo parece estar mal hoy"),
    (1, "Nada me da consuelo hoy"),
    (1, "Siento que todo es una carga"),
    (1, "Estoy atrapado en mis pensamientos"),
    (1, "No tengo energías para nada"),
    (1, "Es un día sin color ni esperanza"),
    (1, "Nada me hace sonreír hoy"),
    (1, "No encuentro alegría en nada"),
    (1, "Es un día de pura tristeza y soledad"),
    (1, "Siento un vacío muy profundo"),
    (1, "Nada tiene sentido hoy"),
    (1, "Me siento como una sombra"),
    (1, "Todo está en mi contra hoy"),
    (1, "Siento que estoy solo"),
    (1, "Es un día sin propósito"),
    (1, "Estoy agobiado por la tristeza"),
    (1, "Siento que me falta algo"),
    (1, "Hoy no encuentro paz"),
    (1, "Es un día para olvidar"),
    (1, "Me siento desconectado del mundo"),
    (1, "Es como si todo pesara más hoy"),
    (1, "No puedo escapar de esta sensación"),
    (1, "Este día solo trae recuerdos amargos"),
    (1, "Me siento incompleto y perdido"),
    (1, "No tengo fuerzas para continuar"),
    (1, "Es un día lleno de desesperanza"),
    (1, "Mi corazón está apesadumbrado"),
    (1, "Este día solo trae desilusión"),

    # Frases de estado de ánimo "Ni fu ni fa"
    (2, "No tengo ganas de hacer nada"),
    (2, "Me siento neutral"),
    (2, "Estoy cansado"),
    (2, "Me da igual lo que pase"),
    (2, "No me importa nada hoy"),
    (2, "Estoy indiferente"),
    (2, "No siento ni alegría ni tristeza"),
    (2, "Hoy es un día normal"),
    (2, "Estoy aburrido"),
    (2, "Nada me parece interesante"),
    (2, "No sé cómo me siento"),
    (2, "Todo está igual que siempre"),
    (2, "Nada especial sucede hoy"),
    (2, "Es un día cualquiera"),
    (2, "Hoy me siento indiferente a todo"),
    (2, "Simplemente no siento nada"),
    (2, "Es uno de esos días sin nada nuevo"),
    (2, "Ni me siento bien ni mal"),
    (2, "Nada me motiva particularmente"),
    (2, "Es un día más, sin nada que contar"),
    (2, "No tengo emociones intensas hoy"),
    (2, "Hoy no pasa nada fuera de lo común"),
    (2, "Es un día bastante monótono"),
    (2, "Todo está en el mismo estado de siempre"),
    (2, "Ni estoy triste ni contento, simplemente estoy"),
    (2, "Hoy no tengo ni energía ni ánimos"),
    (2, "Nada sobresaliente en este día"),
    (2, "Estoy en piloto automático hoy"),
    (2, "Hoy no hay nada que me anime"),
    (2, "Es un día ordinario y sin sorpresas"),
    (2, "Estoy en mi estado habitual"),
    (2, "Hoy no tengo ninguna expectativa"),
    (2, "No espero nada especial hoy"),
    (2, "Es un día como cualquier otro"),
    (2, "Nada parece fuera de lo común"),
    (2, "Hoy solo estoy existiendo"),
    (2, "No hay nada excitante en este día"),
    (2, "Me siento completamente neutral"),
    (2, "Hoy es simplemente un día más"),
    (2, "Nada parece cambiar hoy"),
]

# Preparar X (frases) e y (etiquetas de estados de ánimo)
X = [text.lower() for label, text in dataset]
y = [label for label, text in dataset]

# Generar los embeddings para las frases usando el modelo preentrenado
X_embeddings = model.encode(X)

# Dividir el dataset en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_embeddings, y, test_size=0.2, random_state=42)

# Crear y entrenar el modelo de Regresión Logística
modelo_LR = LogisticRegression(max_iter=1000, multi_class='multinomial', solver='lbfgs', class_weight='balanced')
modelo_LR.fit(X_train, y_train)

# Evaluación del modelo
y_pred_LR = modelo_LR.predict(X_test)
acc_LR = accuracy_score(y_test, y_pred_LR)
report_LR = classification_report(y_test, y_pred_LR, zero_division=1)
print("Precisión Regresión Logística:", acc_LR)
print("Reporte de clasificación Regresión Logística:\n", report_LR)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Precisión Regresión Logística: 0.88
Reporte de clasificación Regresión Logística:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00         8
           1       1.00      0.73      0.84        11
           2       0.67      1.00      0.80         6

    accuracy                           0.88        25
   macro avg       0.89      0.91      0.88        25
weighted avg       0.92      0.88      0.88        25



/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


La precisión obtenida en dicho modelo es del 88%, siendo ese porcentaje de las prediciones realizadas de forma correcta.

Prueba del modelo:

In [11]:
def predict_mood_st(user_input):
    user_input_embedding = model.encode([user_input.lower()])
    mood_label = modelo_LR.predict(user_input_embedding)[0]
    return labels[mood_label][1]


nuevas_frases = [
    "No puedo evitar sentir una gran tristeza hoy",  # Debería ser "Melancólico"
    "Siento que este es uno de los mejores días de mi vida",  # Debería ser "Alegre"
    "Hoy todo me da lo mismo, nada me interesa realmente",  # Debería ser "Ni fu ni fa"
    "No sé qué hacer, pero no estoy ni triste ni feliz",  # Debería ser "Ni fu ni fa"
    "Me siento increíblemente emocionado y lleno de energía",  # Debería ser "Alegre"
    "Todo parece gris y no veo nada que me anime",  # Debería ser "Melancólico"
    "Este día está siendo tan normal como cualquier otro",  # Debería ser "Ni fu ni fa"
    "No hay nada que me entusiasme, es como si estuviera vacío",  # Debería ser "Melancólico"
    "Estoy muy agradecido por todo, este día es fantástico",  # Debería ser "Alegre"
    "Hoy no me pasa nada en particular, no siento ni tristeza ni alegría"  # Debería ser "Ni fu ni fa"
]


for frase in nuevas_frases:
    print(f"La frase '{frase}' tiene el estado de ánimo: {predict_mood_st(frase)}")

La frase 'No puedo evitar sentir una gran tristeza hoy' tiene el estado de ánimo: Melancólico
La frase 'Siento que este es uno de los mejores días de mi vida' tiene el estado de ánimo: Alegre
La frase 'Hoy todo me da lo mismo, nada me interesa realmente' tiene el estado de ánimo: Ni fu ni fa
La frase 'No sé qué hacer, pero no estoy ni triste ni feliz' tiene el estado de ánimo: Ni fu ni fa
La frase 'Me siento increíblemente emocionado y lleno de energía' tiene el estado de ánimo: Alegre
La frase 'Todo parece gris y no veo nada que me anime' tiene el estado de ánimo: Ni fu ni fa
La frase 'Este día está siendo tan normal como cualquier otro' tiene el estado de ánimo: Ni fu ni fa
La frase 'No hay nada que me entusiasme, es como si estuviera vacío' tiene el estado de ánimo: Ni fu ni fa
La frase 'Estoy muy agradecido por todo, este día es fantástico' tiene el estado de ánimo: Alegre
La frase 'Hoy no me pasa nada en particular, no siento ni tristeza ni alegría' tiene el estado de ánimo: Ni fu

Como podemos ver, con SentenceTransformer, el modelo tiene mejores métricas.



Vamos a buscar extraer entidades nombradas de textos en español para que las recomendaciones sean mas precisas.

In [12]:
# Descargar stopwords en español
nltk.download('stopwords')

# Obtener las stopwords en español
spanish_stop_words = stopwords.words('spanish')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [13]:
nlp = spacy.load('es_core_news_md')

def extraer_entidades(texto):
    doc = nlp(texto)
    entidades = [(ent.text, ent.label_) for ent in doc.ents]
    return entidades

# Ejemplo de uso de la función de extracción de entidades
frase_usuario = "Quiero ver una historia de amor en París con personajes como Napoleón"
entidades = extraer_entidades(frase_usuario)
print("Entidades nombradas extraídas:", entidades)


Entidades nombradas extraídas: [('París', 'LOC'), ('Napoleón', 'PER')]


Definimos una función para que cuando los usuarios ingresen sus preferencias o intereses sobre un tema específico se extraigan las entidades.

In [23]:
# Ajustar la función para extraer preferencias y entidades
def ingresar_preferencias(preferencia_usuario):
    entidades = extraer_entidades(preferencia_usuario)
    if entidades:
        print(f"Entidades nombradas detectadas: {entidades}")
    return preferencia_usuario, entidades

**APLICACION**

Para la clasificacion del estado de animo utilizaremos el modelo realizado con SentenceTransformer ya que fue el que mejor metricas nos dio. A su vez,para realizar los embeddings de los datos utilizaremos tambien SentenceTransformers.

In [54]:

model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')


# Procesar los datos y generar embeddings para cada categoría
board_games_df['combined_text'] = (board_games_df['description'] + ' ' + board_games_df['categories']).fillna('')
movies_df['combined_text'] = (movies_df['Description'] + ' ' + movies_df['Genre'] + ' ' + movies_df['Actors']).fillna('')
books_df['combined_text'] = (books_df['Summary'] + ' ' + books_df['Author'] + '' + books_df['Subjects']).fillna('')

board_games_df['embeddings'] = list(model.encode(board_games_df['combined_text'].tolist()))
movies_df['embeddings'] = list(model.encode(movies_df['combined_text'].tolist()))
books_df['embeddings'] = list(model.encode(books_df['combined_text'].tolist()))



def recomendar(input_estado_animo, estado_de_animo, input_tematica):
    # Extraer entidades de la temática ingresada por el usuario
    input_tematica, entidades = ingresar_preferencias(input_tematica)

    # Crear una frase de entrada más completa con las entidades detectadas
    total_input = input_estado_animo + ' ' + estado_de_animo + ' ' + input_tematica
    if entidades:
        total_input += ' ' + ' '.join([ent[0] for ent in entidades])

    print(f"Buscando recomendaciones para: {input_tematica} con contexto adicional de entidades")

    # Generar el embedding de la frase de entrada
    vector_comparar = model.encode(total_input)

    # Calcular similitudes para cada dataset
    similitudes_juegos = cosine_similarity([vector_comparar], board_games_df['embeddings'].tolist())
    similitudes_peliculas = cosine_similarity([vector_comparar], movies_df['embeddings'].tolist())
    similitudes_libros = cosine_similarity([vector_comparar], books_df['embeddings'].tolist())

    # Obtener los índices de las tres recomendaciones más similares para cada categoría
    top_indices_juegos = similitudes_juegos[0].argsort()[-3:][::-1]
    top_indices_peliculas = similitudes_peliculas[0].argsort()[-3:][::-1]
    top_indices_libros = similitudes_libros[0].argsort()[-3:][::-1]

    # Crear diccionarios con las recomendaciones
    recomendaciones_juegos = [
        {'Titulo': board_games_df.iloc[index]['game_name'], 'Summary': limpiar_descripcion(board_games_df.iloc[index]['description'])}
        for index in top_indices_juegos
    ]
    recomendaciones_peliculas = [
        {'Titulo': movies_df.iloc[index]['Title'], 'Summary': movies_df.iloc[index]['Description']}
        for index in top_indices_peliculas
    ]
    recomendaciones_libros = [
        {'Titulo': books_df.iloc[index]['Title'], 'Summary': books_df.iloc[index]['Summary']}
        for index in top_indices_libros
    ]

    # Consolidar y devolver las recomendaciones
    return {
        'juegos': recomendaciones_juegos,
        'peliculas': recomendaciones_peliculas,
        'libros': recomendaciones_libros
    }

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [55]:
def mostrar_interfaz():
    # Crear entradas de usuario para el estado de ánimo y preferencia temática
    estado_animo_input = widgets.Text(
        value='',
        placeholder='¿Cómo te sientes hoy?',
        description='Estado de Ánimo:',
        disabled=False
    )

    preferencia_input = widgets.Text(
        value='',
        placeholder='Describe la temática que te gustaría explorar',
        description='Preferencia:',
        disabled=False
    )

    boton_enviar = widgets.Button(description="Obtener Recomendaciones")
    output = widgets.Output()

    # Función que maneja el clic en el botón de recomendación
    def on_button_clicked(b):
        with output:
            output.clear_output()  # Limpiar el área de salida antes de mostrar nuevos resultados

            # Obtener valores de estado de ánimo y preferencia del usuario desde los widgets
            estado_animo = estado_animo_input.value
            preferencia = preferencia_input.value

            # Verificar si hay valores en los campos
            if not estado_animo or not preferencia:
                print("Por favor ingresa tanto el estado de ánimo como la preferencia.")
                return

            # Detectar el estado de ánimo usando predict_mood_st
            estado_detectado = predict_mood_st(estado_animo)
            print(f"Estado de ánimo detectado: {estado_detectado}")

            # Generar recomendaciones combinando estado de ánimo detectado y preferencia
            recomendaciones = recomendar(estado_animo, estado_detectado, preferencia)

            # Mostrar las recomendaciones en la interfaz
            print("\nRecomendaciones de juegos:")
            for rec in recomendaciones['juegos']:
                print(f"- {rec['Titulo']}: {rec['Summary']}")

            print("\nRecomendaciones de películas:")
            for rec in recomendaciones['peliculas']:
                print(f"- {rec['Titulo']}: {rec['Summary']}")

            print("\nRecomendaciones de libros:")
            for rec in recomendaciones['libros']:
                print(f"- {rec['Titulo']}: {rec['Summary']}")

    # Conectar el evento de clic del botón con la función on_button_clicked
    boton_enviar.on_click(on_button_clicked)

    # Mostrar los widgets en la interfaz
    display(estado_animo_input, preferencia_input, boton_enviar, output)

# Ejecutar la interfaz
mostrar_interfaz()

Text(value='', description='Estado de Ánimo:', placeholder='¿Cómo te sientes hoy?')

Text(value='', description='Preferencia:', placeholder='Describe la temática que te gustaría explorar')

Button(description='Obtener Recomendaciones', style=ButtonStyle())

Output()